#**1. Algorithme génétique**
##1.1 Approche naive


# variante 1 fonction d'adaptation nb conflit + nb de couleurs

In [ ]:
import random
import time
import numpy as np

class GeneticGraphColoring:
    def __init__(self, adjacency_matrix):
        self.graph = adjacency_matrix
        self.num_nodes = len(adjacency_matrix)
        self.max_colors = self.num_nodes
        self.best_coloring = None
        self.best_colors_used = self.num_nodes

    # Fonction d'adaptation (fitness) : nombre de conflits + nombre de couleurs
    def fitness(self, individual):
        conflicts = 0
        used_colors = len(set(individual))
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                if self.graph[i][j] == 1 and individual[i] == individual[j]:
                    conflicts += 1
        return conflicts * 100 + used_colors

    # Initialisation aléatoire de la population
    def initialize_population(self, size):
        return [[random.randint(0, self.max_colors - 1) for _ in range(self.num_nodes)] for _ in range(size)]

    # Sélection par rang
    def rank_selection(self, population):
        population.sort(key=lambda ind: self.fitness(ind))
        return population[:self.intermediate_size]

    # Croisement à un point
    def crossover(self, parent1, parent2):
        point = random.randint(1, self.num_nodes - 2)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

    # Mutation sur un sommet aléatoire
    def mutate(self, individual):
        mutated = individual[:]
        node = random.randint(0, self.num_nodes - 1)
        current_color = mutated[node]
        available_colors = [c for c in range(self.max_colors) if c != current_color]
        if available_colors:
            mutated[node] = random.choice(available_colors)
        return mutated

    # Algorithme génétique principal
    def run(self, pop_size, intermediate_size, crossover_rate, mutation_rate, max_iter, stagnation_threshold):

        self.intermediate_size = intermediate_size
        population = self.initialize_population(pop_size) # population initiale
        stagnation = 0
        best_fitness = float('inf')
        start_time = time.time()

        for iteration in range(max_iter):
            # 1. Sélection
            intermediate = self.rank_selection(population) # on selectionne les individus avec meilleures fitness
            offspring = intermediate[:] # offspring : population intermediaire .

            # 2. Croisement
            for _ in range(len(intermediate) // 2): # on prend a chaque it deux parents
                if random.random() < crossover_rate: # est ce que on fait croisement ou pas
                    p1, p2 = random.sample(intermediate, 2)
                    c1, c2 = self.crossover(p1, p2)
                    offspring.extend([c1, c2]) # on rajoute a la population intermediaire les nouvelles sol generees par croisement

            # 3. Mutation
            for i in range(len(offspring)): # parents + fils
                if random.random() < mutation_rate: # on verifie pour chaque individu est ce que on fait de la mutation ou pas
                    offspring[i] = self.mutate(offspring[i])

            # 4. Mise à jour de la population
            offspring.sort(key=lambda ind: self.fitness(ind)) # on prend les meilleurs solutions resultantes du croisement et mutation
            population = offspring[:pop_size]

            # 5. Mise à jour du meilleur individu , on cherche est ce que nous avons une meilleure solution dans cette population .
            current_best = population[0] # car population est deja ordonnee par fitness
            current_fitness = self.fitness(current_best)
            current_conflicts = sum(1 for i in range(self.num_nodes) for j in range(i + 1, self.num_nodes)
                                    if self.graph[i][j] == 1 and current_best[i] == current_best[j])
            current_colors = len(set(current_best))

            if current_conflicts == 0 and current_colors < self.best_colors_used:
                self.best_coloring = current_best[:]
                self.best_colors_used = current_colors
                stagnation = 0
            else:
                stagnation += 1

            if stagnation >= stagnation_threshold:
                break

        end_time = time.time()
        return self.best_coloring, self.best_colors_used, end_time - start_time

# Lecture du fichier DIMACS

def read_dimacs_graph(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    edges = []
    num_nodes = 0
    for line in lines:
        if line.startswith('p'):
            _, _, n, _ = line.strip().split()
            num_nodes = int(n)
        elif line.startswith('e'):
            _, u, v = line.strip().split()
            edges.append((int(u) - 1, int(v) - 1))

    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)
    for u, v in edges:
        adjacency_matrix[u, v] = 1
        adjacency_matrix[v, u] = 1

    print("Nombre de sommets détectés :", num_nodes)
    print("Arêtes détectées :", edges[:10])
    return adjacency_matrix

# Fonction principale

def main():
    file_path = "/content/dsjc250.5.col"
    adjacency_matrix = read_dimacs_graph(file_path)

    ggc = GeneticGraphColoring(adjacency_matrix)
    best_coloring, min_colors, exec_time = ggc.run(
        pop_size=100,
        intermediate_size=50,
        crossover_rate=0.8,
        mutation_rate=0.1,
        max_iter=1000,
        stagnation_threshold=100
    )

    print("Meilleure coloration:", best_coloring)
    print("Nombre minimal de couleurs utilisées:", min_colors)
    print("Temps d'exécution:", exec_time, "secondes")

if __name__ == "__main__":
    main()


# Variante 2  fonction d'adaptation nb de couleurs uniquement (n'autorise pas la générationde solutions conflictuelles)

In [ ]:
import random
import time
import numpy as np

class GeneticGraphColoringNoConflict:
    def __init__(self, adjacency_matrix):
        self.graph = adjacency_matrix
        self.num_nodes = len(adjacency_matrix)
        self.max_colors = self.num_nodes
        self.best_coloring = None
        self.best_colors_used = self.num_nodes



    # Fonction pour vérifie s'il y a des conflits dans une solution
    def has_conflicts(self, individual):
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                if self.graph[i][j] == 1 and individual[i] == individual[j]:
                    return True
        return False



    # Fonction d'adaptation : considère nombre de couleurs utilisées uniquement tant que on accepte pas les conflits lors de la génération de solutions
    def fitness(self, individual):
        return len(set(individual))



    # Fonction qui permet de générer une solution sans conflit pour l'utiliser dans la génération de la population initiale sans conflits
    def generate_valid_individual(self):
        individual = [-1] * self.num_nodes
        for node in range(self.num_nodes):
            neighbor_colors = {individual[neighbor] for neighbor in range(self.num_nodes) if self.graph[node][neighbor] == 1 and individual[neighbor] != -1}
            available_colors = [c for c in range(self.max_colors) if c not in neighbor_colors]
            if available_colors:
                individual[node] = random.choice(available_colors)
            else:
                # Si plus de couleur disponible, on rajoute une nouvelle couleur
                individual[node] = max(neighbor_colors, default=-1) + 1
        return individual



    # Générer la population intiale
    def initialize_population(self, size):
        population = []
        while len(population) < size:
            individual = self.generate_valid_individual()
            population.append(individual)
        return population


    # Sélection par rang : on ordonne les solutions de la population selon la fitness puis on prend les meilleures solutions  (intermediate_size)
    def rank_selection(self, population):
        population.sort(key=lambda ind: self.fitness(ind))
        return population[:self.intermediate_size]


    # Croisement qui génère seulement des enfants valides (solutions sans conflits)
    def crossover(self, parent1, parent2):
        for _ in range(5):  # On tente 5 fois de générer des fils sans conflits des parents choisi a chaque fois on essaye un point de croisement diff
            point = random.randint(1, self.num_nodes - 2) # Le point de croisement
            child1 = parent1[:point] + parent2[point:]
            child2 = parent2[:point] + parent1[point:]
            if not self.has_conflicts(child1) and not self.has_conflicts(child2):
                return child1, child2
        # Si échec, retourner les parents inchangés
        return parent1[:], parent2[:]



    # Mutation qui génère seulement un individu valide (sans conflit)
    def mutate(self, individual):
        for _ in range(5):  # Tenter 5 mutations valides , a chaque fois on tente avec un nouveau noeud pour la mutation
            mutated = individual[:]
            node = random.randint(0, self.num_nodes - 1)
            neighbor_colors = {mutated[neighbor] for neighbor in range(self.num_nodes) if self.graph[node][neighbor] == 1}
            available_colors = [c for c in range(self.max_colors) if c not in neighbor_colors]
            if available_colors:
                mutated[node] = random.choice(available_colors)
                if not self.has_conflicts(mutated):
                    return mutated
        return individual[:]  # Si mutation échoue, garder l'individu original



    # Algorithme génétique principal
    def run(self, pop_size, intermediate_size, crossover_rate, mutation_rate, max_iter, stagnation_threshold):
        self.intermediate_size = intermediate_size
        # initialiser la population
        population = self.initialize_population(pop_size)

        stagnation = 0
        best_fitness = float('inf')
        start_time = time.time()

        for iteration in range(max_iter):

            # 1. Sélection
            intermediate = self.rank_selection(population)
            offspring = intermediate[:]

            # 2. Croisement
            for _ in range(len(intermediate) // 2):
                if random.random() < crossover_rate:
                    p1, p2 = random.sample(intermediate, 2)
                    c1, c2 = self.crossover(p1, p2)
                    offspring.extend([c1, c2])

            # 3. Mutation
            for i in range(len(offspring)):
                if random.random() < mutation_rate:
                    offspring[i] = self.mutate(offspring[i])

            # 4. Mise à jour de la population
            offspring.sort(key=lambda ind: self.fitness(ind))
            population = offspring[:pop_size]

            # 5. Mise à jour du meilleur individu
            current_best = population[0]
            current_fitness = self.fitness(current_best)

            if current_fitness < self.best_colors_used:
                self.best_coloring = current_best[:]
                self.best_colors_used = current_fitness
                stagnation = 0
            else:
                stagnation += 1

            if stagnation >= stagnation_threshold:
                break

        end_time = time.time()
        return self.best_coloring, self.best_colors_used, end_time - start_time

# Lecture du fichier DIMACS

def read_dimacs_graph(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    edges = []
    num_nodes = 0
    for line in lines:
        if line.startswith('p'):
            _, _, n, _ = line.strip().split()
            num_nodes = int(n)
        elif line.startswith('e'):
            _, u, v = line.strip().split()
            edges.append((int(u) - 1, int(v) - 1))

    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)
    for u, v in edges:
        adjacency_matrix[u, v] = 1
        adjacency_matrix[v, u] = 1

    print("Nombre de sommets détectés :", num_nodes)
    print("Arêtes détectées :", edges[:10])
    return adjacency_matrix

# Fonction principale

def main():
    file_path = "/content/r250.5.col.txt"
    adjacency_matrix = read_dimacs_graph(file_path)

    ggc = GeneticGraphColoringNoConflict(adjacency_matrix)
    best_coloring, min_colors, exec_time = ggc.run(
        pop_size=100,
        intermediate_size=50,
        crossover_rate=0.8,
        mutation_rate=0.1,
        max_iter=1000,
        stagnation_threshold=100
    )

    print("Meilleure coloration:", best_coloring)
    print("Nombre minimal de couleurs utilisées:", min_colors)
    print("Temps d'exécution:", exec_time, "secondes")

if __name__ == "__main__":
    main()


Nombre de sommets détectés : 250
Arêtes détectées : [(2, 0), (2, 1), (3, 0), (4, 0), (4, 1), (5, 0), (5, 1), (5, 2), (5, 4), (6, 1)]
Meilleure coloration: [215, 186, 184, 55, 87, 211, 163, 179, 184, 223, 210, 186, 165, 207, 241, 190, 127, 189, 210, 83, 24, 25, 164, 139, 98, 174, 29, 105, 32, 240, 140, 112, 173, 127, 227, 29, 164, 26, 14, 17, 33, 126, 135, 155, 7, 14, 21, 227, 18, 115, 77, 85, 175, 34, 99, 191, 189, 74, 47, 206, 101, 148, 115, 23, 152, 30, 188, 30, 215, 101, 175, 66, 245, 17, 17, 73, 118, 238, 55, 57, 35, 149, 65, 160, 133, 87, 211, 112, 209, 177, 210, 118, 0, 1, 47, 29, 232, 174, 148, 126, 21, 83, 212, 18, 59, 224, 122, 93, 212, 173, 206, 193, 23, 30, 211, 126, 241, 180, 234, 103, 100, 171, 152, 24, 1, 154, 112, 72, 86, 116, 14, 171, 173, 200, 148, 218, 61, 208, 224, 8, 160, 54, 128, 44, 180, 92, 1, 77, 8, 163, 127, 59, 93, 188, 160, 208, 178, 77, 65, 90, 5, 179, 193, 34, 238, 18, 135, 218, 213, 14, 12, 78, 238, 12, 22, 163, 93, 154, 103, 100, 128, 61, 164, 133, 100, 2

##1.2 Approche à deux phases

In [ ]:
import numpy as np
import random
import time

################################################################################

def read_dimacs_graph(file_path):
    """Lit un graphe au format DIMACS et retourne sa matrice d'adjacence."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    edges = []
    num_nodes = 0

    for line in lines:
        line = line.strip()
        if line.startswith('p'):  # Problem line
            parts = line.split()
            num_nodes = int(parts[2])
        elif line.startswith('e'):  # Edge line
            parts = line.split()
            # DIMACS vertices are 1-indexed, convert to 0-indexed
            node1, node2 = int(parts[1]) - 1, int(parts[2]) - 1
            edges.append((node1, node2))

    # Create the adjacency matrix
    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    # Fill the adjacency matrix
    for node1, node2 in edges:
        adjacency_matrix[node1, node2] = 1
        adjacency_matrix[node2, node1] = 1  # Undirected graph

    return adjacency_matrix

################################################################################

class TwoPhaseGeneticAlgorithmGraphColoring:
    def __init__(self, adjacency_matrix, population_size=100, max_generations=500,
                 crossover_rate=0.8, mutation_rate=0.2, tournament_size=3, elitism=True):
        """
        Initialise l'algorithme génétique pour la coloration de graphe.

        Args:
            adjacency_matrix: Matrice d'adjacence du graphe
            population_size: Taille de la population
            max_generations: Nombre maximal de générations
            crossover_rate: Taux de croisement
            mutation_rate: Taux de mutation
            tournament_size: Taille du tournoi pour la sélection
            elitism: Si True, conserve le meilleur individu de chaque génération
        """
        self.adjacency_matrix = adjacency_matrix
        self.num_nodes = adjacency_matrix.shape[0]
        self.population_size = population_size
        self.max_generations = max_generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.tournament_size = tournament_size
        self.elitism = elitism

        # Calculer le degré maximal du graphe pour estimer le nombre initial de couleurs
        degrees = np.sum(adjacency_matrix, axis=1)
        self.max_degree = np.max(degrees)
        # Une borne supérieure du nombre chromatique est max_degree + 1
        self.initial_max_colors = self.max_degree + 1

        # Phase actuelle: 1 = trouver coloration valide, 2 = minimiser les couleurs
        self.current_phase = 1

    def initialize_population(self):
        """Initialise une population de solutions aléatoires."""
        population = []
        for _ in range(self.population_size):
            # Chaque individu est une liste où l'indice représente le sommet
            # et la valeur représente la couleur (de 0 à initial_max_colors-1)
            individual = [random.randint(0, self.initial_max_colors - 1) for _ in range(self.num_nodes)]
            population.append(individual)
        return population

    def calculate_conflicts(self, individual):
        """Calcule le nombre de conflits dans une coloration."""
        conflicts = 0
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                if self.adjacency_matrix[i, j] == 1 and individual[i] == individual[j]:
                    conflicts += 1
        return conflicts

    def calculate_fitness(self, individual):
        """
        Calcule la valeur d'adaptation d'un individu selon la phase actuelle.

        Phase 1: On se concentre uniquement sur l'élimination des conflits
        Phase 2: On minimise le nombre de couleurs tout en maintenant une coloration valide
        """
        conflicts = self.calculate_conflicts(individual)

        # Compter les couleurs utilisées
        num_colors_used = len(set(individual))

        # Fonction fitness selon la phase
        if self.current_phase == 1:
            # Phase 1: Eliminer les conflits est la priorité
            fitness = conflicts * 1000
        else:
            # Phase 2: La coloration est valide, minimiser le nombre de couleurs
            if conflicts > 0:
                # Si des conflits apparaissent en phase 2, les pénaliser fortement
                fitness = conflicts * 10000
            else:
                fitness = num_colors_used

        return fitness

    def is_valid_coloring(self, individual):
        """Vérifie si la coloration respecte la contrainte qu'aucun sommet adjacent n'a la même couleur."""
        return self.calculate_conflicts(individual) == 0

    def tournament_selection(self, population, fitnesses):
        """Sélectionne un individu par tournoi."""
        selected_indices = random.sample(range(len(population)), self.tournament_size)
        selected_fitness = [fitnesses[i] for i in selected_indices]
        # Sélectionner l'individu avec la meilleure fitness (la plus petite valeur)
        return population[selected_indices[selected_fitness.index(min(selected_fitness))]]

    def crossover(self, parent1, parent2):
        """
        Effectue un croisement entre deux parents pour créer deux enfants.
        Utilise le croisement à un point.
        """
        if random.random() < self.crossover_rate:
            crossover_point = random.randint(1, self.num_nodes - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            return child1, child2
        else:
            return parent1.copy(), parent2.copy()

    def mutation(self, individual):
        """
        Applique une mutation à un individu selon la phase actuelle.
        """
        mutated = individual.copy()

        # En phase 1, permettre plus de diversité pour résoudre les conflits
        if self.current_phase == 1:
            max_color = max(individual)

            for i in range(self.num_nodes):
                if random.random() < self.mutation_rate:
                    # Identifier les conflits pour ce sommet
                    has_conflict = False
                    conflicting_colors = set()

                    for j in range(self.num_nodes):
                        if self.adjacency_matrix[i, j] == 1 and individual[i] == individual[j]:
                            has_conflict = True
                            conflicting_colors.add(individual[j])

                    # Si ce sommet a un conflit, choisir une couleur non conflictuelle
                    if has_conflict:
                        existing_colors = set(individual)
                        # Essayer de trouver une couleur existante non conflictuelle
                        valid_colors = existing_colors - conflicting_colors

                        if valid_colors:
                            mutated[i] = random.choice(list(valid_colors))
                        else:
                            # Utiliser une nouvelle couleur
                            mutated[i] = max_color + 1
                    else:
                        # Pas de conflit, mutation normale
                        if random.random() < 0.3:  # 30% de chance de changer
                            mutated[i] = random.randint(0, max_color)
        else:
            # En phase 2, essayer de réduire le nombre de couleurs
            used_colors = set(individual)
            color_frequency = {color: individual.count(color) for color in used_colors}
            min_color = min(used_colors)

            for i in range(self.num_nodes):
                if random.random() < self.mutation_rate:
                    current_color = individual[i]

                    # Identifier les couleurs utilisées par les voisins
                    neighbor_colors = set()
                    for j in range(self.num_nodes):
                        if self.adjacency_matrix[i, j] == 1:
                            neighbor_colors.add(individual[j])

                    # Chercher la couleur la plus basse non utilisée par les voisins
                    valid_colors = []
                    for color in range(min_color, max(used_colors) + 1):
                        if color not in neighbor_colors:
                            valid_colors.append(color)

                    if valid_colors:
                        # Préférer les couleurs existantes avec une fréquence élevée
                        valid_existing_colors = [c for c in valid_colors if c in used_colors]
                        if valid_existing_colors:
                            # Choisir une couleur existante pour réduire la diversité des couleurs
                            weights = [color_frequency.get(c, 0) for c in valid_existing_colors]
                            total = sum(weights)
                            if total > 0:
                                weights = [w/total for w in weights]
                                mutated[i] = np.random.choice(valid_existing_colors, p=weights)
                            else:
                                mutated[i] = random.choice(valid_existing_colors)
                        else:
                            # Sinon utiliser la couleur la plus basse disponible
                            mutated[i] = min(valid_colors)

        return mutated

    def optimize_colors(self, individual):
        """
        Optimise la coloration en réduisant les numéros de couleur si possible.
        Réindexe les couleurs de manière consécutive.
        """
        unique_colors = set(individual)
        color_map = {old_color: new_color for new_color, old_color in enumerate(sorted(unique_colors))}
        return [color_map[color] for color in individual]

    def local_search(self, individual):
        """
        Améliore la solution en cherchant localement à réduire les conflits (phase 1)
        ou le nombre de couleurs (phase 2).
        """
        improved = individual.copy()

        if self.current_phase == 1:
            # En phase 1, essayer de résoudre les conflits
            for i in range(self.num_nodes):
                # Vérifier si ce sommet a des conflits
                has_conflict = False
                for j in range(self.num_nodes):
                    if self.adjacency_matrix[i, j] == 1 and improved[i] == improved[j]:
                        has_conflict = True
                        break

                if has_conflict:
                    # Collecter les couleurs des voisins
                    neighbor_colors = set()
                    for j in range(self.num_nodes):
                        if self.adjacency_matrix[i, j] == 1:
                            neighbor_colors.add(improved[j])

                    # Trouver une couleur non conflictuelle
                    current_colors = set(improved)
                    for color in range(len(current_colors) + 1):
                        if color not in neighbor_colors:
                            improved[i] = color
                            break
        else:
            # En phase 2, essayer de réduire les couleurs
            colors_used = set(improved)

            # Trier les couleurs par fréquence d'utilisation (commencer par les moins fréquentes)
            color_freq = {color: improved.count(color) for color in colors_used}
            sorted_colors = sorted(color_freq.items(), key=lambda x: x[1])

            # Pour chaque couleur (en commençant par les moins fréquentes)
            for color, _ in sorted_colors:
                # Pour chaque sommet avec cette couleur
                for i in range(self.num_nodes):
                    if improved[i] == color:
                        # Collecter les couleurs des voisins
                        neighbor_colors = set()
                        for j in range(self.num_nodes):
                            if self.adjacency_matrix[i, j] == 1:
                                neighbor_colors.add(improved[j])

                        # Essayer les couleurs existantes avec une fréquence plus élevée
                        for other_color, freq in reversed(sorted_colors):
                            if other_color != color and other_color not in neighbor_colors:
                                improved[i] = other_color
                                break

        return improved

    def run(self, verbose=True):
        """
        Exécute l'algorithme génétique en deux phases.

        Returns:
            best_individual: Meilleure solution trouvée
            num_colors: Nombre de couleurs utilisées dans la meilleure solution
            is_valid: True si la coloration est valide, False sinon
        """
        start_time = time.time()

        # Phase 1: Trouver une coloration valide
        self.current_phase = 1
        print("Phase 1: Recherche d'une coloration valide...")

        # Initialiser la population
        population = self.initialize_population()

        best_individual = None
        best_fitness = float('inf')
        best_phase1_individual = None

        for generation in range(self.max_generations):
            # Calculer la fitness pour chaque individu
            fitnesses = [self.calculate_fitness(ind) for ind in population]

            # Trouver le meilleur individu
            current_best_index = fitnesses.index(min(fitnesses))
            current_best = population[current_best_index]
            current_best_fitness = fitnesses[current_best_index]

            if current_best_fitness < best_fitness:
                best_fitness = current_best_fitness
                best_individual = current_best.copy()

                # Optimiser la coloration
                best_individual = self.optimize_colors(best_individual)
                conflicts = self.calculate_conflicts(best_individual)
                num_colors = len(set(best_individual))
                is_valid = conflicts == 0

                if verbose and (generation % 10 == 0 or generation == self.max_generations - 1):
                    elapsed_time = time.time() - start_time
                    print(f"Génération {generation}: Meilleure fitness = {best_fitness}, "
                          f"Conflits = {conflicts}, Couleurs = {num_colors}, "
                          f"Valide = {is_valid}, Temps écoulé = {elapsed_time:.2f}s")

                # Si on a trouvé une coloration valide, passer à la phase 2
                if is_valid:
                    best_phase1_individual = best_individual.copy()
                    break

            # Créer la nouvelle génération
            new_population = []

            # Élitisme: garder le meilleur individu
            if self.elitism:
                new_population.append(current_best)

            # Créer le reste de la population par sélection, croisement et mutation
            while len(new_population) < self.population_size:
                parent1 = self.tournament_selection(population, fitnesses)
                parent2 = self.tournament_selection(population, fitnesses)

                child1, child2 = self.crossover(parent1, parent2)

                child1 = self.mutation(child1)
                child1 = self.local_search(child1)

                child2 = self.mutation(child2)
                child2 = self.local_search(child2)

                new_population.append(child1)
                if len(new_population) < self.population_size:
                    new_population.append(child2)

            population = new_population

        # Si on n'a pas trouvé de coloration valide en phase 1
        if not is_valid:
            print("Phase 1 n'a pas trouvé de coloration valide.")
            return best_individual, num_colors, is_valid

        # Phase 2: Minimiser le nombre de couleurs
        self.current_phase = 2
        print("\nPhase 2: Minimisation du nombre de couleurs...")

        # Réinitialiser la population avec le meilleur individu de la phase 1
        population = [best_phase1_individual.copy() for _ in range(self.population_size)]
        # Appliquer de légères mutations à chaque individu sauf le premier pour assurer la diversité
        for i in range(1, self.population_size):
            population[i] = self.mutation(population[i])
            population[i] = self.local_search(population[i])
            population[i] = self.optimize_colors(population[i])

        best_fitness = self.calculate_fitness(best_phase1_individual)
        best_individual = best_phase1_individual.copy()
        best_num_colors = len(set(best_individual))

        for generation in range(self.max_generations):
            # Calculer la fitness pour chaque individu
            fitnesses = [self.calculate_fitness(ind) for ind in population]

            # Trouver le meilleur individu
            current_best_index = fitnesses.index(min(fitnesses))
            current_best = population[current_best_index]
            current_best_fitness = fitnesses[current_best_index]

            # Optimiser la coloration
            current_best = self.optimize_colors(current_best)

            if current_best_fitness <= best_fitness:  # <= pour favoriser les solutions avec moins de couleurs
                best_fitness = current_best_fitness

                current_conflicts = self.calculate_conflicts(current_best)
                current_num_colors = len(set(current_best))

                # Ne mettre à jour que si la coloration est valide et utilise moins de couleurs
                if current_conflicts == 0 and current_num_colors <= best_num_colors:
                    best_individual = current_best.copy()
                    best_num_colors = current_num_colors

                    if verbose and (generation % 10 == 0 or generation == self.max_generations - 1):
                        elapsed_time = time.time() - start_time
                        print(f"Génération {generation}: Couleurs = {best_num_colors}, "
                              f"Valide = {current_conflicts == 0}, Temps écoulé = {elapsed_time:.2f}s")

            # Créer la nouvelle génération
            new_population = []

            # Élitisme: garder le meilleur individu
            if self.elitism:
                new_population.append(current_best)

            # Créer le reste de la population
            while len(new_population) < self.population_size:
                parent1 = self.tournament_selection(population, fitnesses)
                parent2 = self.tournament_selection(population, fitnesses)

                child1, child2 = self.crossover(parent1, parent2)

                child1 = self.mutation(child1)
                child1 = self.local_search(child1)
                child1 = self.optimize_colors(child1)

                child2 = self.mutation(child2)
                child2 = self.local_search(child2)
                child2 = self.optimize_colors(child2)

                new_population.append(child1)
                if len(new_population) < self.population_size:
                    new_population.append(child2)

            population = new_population

        # Optimiser la coloration finale
        best_individual = self.optimize_colors(best_individual)
        num_colors = len(set(best_individual))
        is_valid = self.is_valid_coloring(best_individual)

        elapsed_time = time.time() - start_time
        if verbose:
            print(f"\nRésultats après {generation+1} générations de la phase 2:")
            print(f"Nombre de couleurs: {num_colors}")
            print(f"Coloration valide: {is_valid}")
            print(f"Temps total d'exécution: {elapsed_time:.2f} secondes")

        return best_individual, num_colors, is_valid

# Exemple d'utilisation
if __name__ == "__main__":
    # Charger un graphe depuis un fichier DIMACS
    file_path = "/content/r250.5.col.txt"
    adjacency_matrix = read_dimacs_graph(file_path)

    # Paramètres de l'algorithme génétique
    population_size = 100
    max_generations = 300  # Pour chaque phase
    crossover_rate = 0.8
    mutation_rate = 0.2
    tournament_size = 3
    elitism = True

    # Créer et exécuter l'algorithme génétique
    ga = TwoPhaseGeneticAlgorithmGraphColoring(
        adjacency_matrix,
        population_size=population_size,
        max_generations=max_generations,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        tournament_size=tournament_size,
        elitism=elitism
    )

    best_solution, num_colors, is_valid = ga.run(verbose=True)

    print("\nMeilleure solution trouvée:")
    print(f"Coloration: {best_solution}")
    print(f"Nombre de couleurs: {num_colors}")
    print(f"Coloration valide: {is_valid}")

Phase 1: Recherche d'une coloration valide...
Génération 0: Meilleure fitness = 56000, Conflits = 56, Couleurs = 149, Valide = False, Temps écoulé = 0.69s

Phase 2: Minimisation du nombre de couleurs...
Génération 0: Couleurs = 85, Valide = True, Temps écoulé = 8.18s
Génération 10: Couleurs = 74, Valide = True, Temps écoulé = 43.85s
Génération 20: Couleurs = 72, Valide = True, Temps écoulé = 76.84s
Génération 30: Couleurs = 72, Valide = True, Temps écoulé = 110.29s
Génération 40: Couleurs = 72, Valide = True, Temps écoulé = 143.63s
Génération 50: Couleurs = 71, Valide = True, Temps écoulé = 175.97s
Génération 60: Couleurs = 71, Valide = True, Temps écoulé = 209.20s
Génération 70: Couleurs = 71, Valide = True, Temps écoulé = 242.37s
Génération 80: Couleurs = 71, Valide = True, Temps écoulé = 274.57s
Génération 90: Couleurs = 71, Valide = True, Temps écoulé = 307.82s
Génération 100: Couleurs = 71, Valide = True, Temps écoulé = 340.80s
Génération 110: Couleurs = 71, Valide = True, Temps é

##1.2.2 Optuna


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.8 MB/s eta 0:00:00


In [ ]:
import optuna

def objective(trial):
    # Hyperparamètres à optimiser
    crossover_rate = trial.suggest_float("crossover_rate", 0.6, 0.95)
    mutation_rate = trial.suggest_float("mutation_rate", 0.01, 0.5)
    tournament_size = trial.suggest_int("tournament_size", 2, 10)
    elitism = trial.suggest_categorical("elitism", [True, False])

    # Lecture du graphe (mets bien le fichier sur ton Drive ou dans Colab)
    file_path = "dsjc250.5.col"
    adjacency_matrix = read_dimacs_graph(file_path)

    # Lancer l’algorithme génétique avec les paramètres proposés
    ga = TwoPhaseGeneticAlgorithmGraphColoring(
        adjacency_matrix,
        population_size=100,
        max_generations=100,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        tournament_size=tournament_size,
        elitism=elitism
    )

    best_individual, num_colors, is_valid = ga.run(verbose=False)

    # On veut minimiser le nombre de couleurs si la solution est valide
    if is_valid:
        return num_colors
    else:
        # Pénaliser les solutions non valides
        return float('inf')


In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=3)

# Résultat
print("Meilleurs hyperparamètres trouvés :")
print(study.best_params)
print(f"Nombre de couleurs obtenu : {study.best_value}")


#2. COLONIES DE FOURMIS (Algorithme 1)

In [ ]:
import random
import time
from collections import defaultdict, Counter

def lire_graphe(fichier):
    """Lit un graphe depuis un fichier au format DIMACS"""
    graphe = {}
    with open(fichier, 'r') as f:
        for ligne in f:
            if ligne.startswith('e'):
                _, u, v = ligne.split()
                u, v = int(u), int(v)
                if u not in graphe:
                    graphe[u] = []
                if v not in graphe:
                    graphe[v] = []
                graphe[u].append(v)
                graphe[v].append(u)
    return graphe

def initialiser_pheromones(graphe, valeur_initiale=1.0):
    """Initialise les traces de phéromones sur toutes les arêtes"""
    return {u: {v: valeur_initiale for v in graphe[u]} for u in graphe}

def degre_saturation(solution, graphe, noeud):
    """Calcule le degré de saturation (nombre de couleurs voisines différentes)"""
    couleurs_voisines = {solution[v] for v in graphe[noeud] if v in solution}
    return len(couleurs_voisines)

def couleurs_disponibles(solution, graphe, noeud, max_couleurs):
    """Retourne les couleurs disponibles pour un noeud"""
    couleurs_interdites = {solution[v] for v in graphe[noeud] if v in solution}
    return [c for c in range(max_couleurs) if c not in couleurs_interdites]

def construire_solution(graphe, pheromones, alpha, beta):
    """Construit une solution de coloration pour une fourmi"""
    solution = {}
    noeuds = list(graphe.keys())

    # Tri initial par degré décroissant
    noeuds.sort(key=lambda n: -len(graphe[n]))
    solution[noeuds[0]] = 0
    max_couleurs = 1

    noeuds_restants = noeuds[1:]

    while noeuds_restants:
        # Tri par degré de saturation - CORRECTION ICI
        noeuds_restants.sort(key=lambda n: (
            -degre_saturation(solution, graphe, n),
            -len(graphe[n])
        ))  # Cette parenthèse ferme le sort()

        noeud_courant = noeuds_restants.pop(0)

        # Calcul des probabilités
        couleurs = couleurs_disponibles(solution, graphe, noeud_courant, max_couleurs)
        if not couleurs:
            couleurs = [max_couleurs]
            max_couleurs += 1

        probabilites = []
        for couleur in couleurs:
            # Calcul heuristique
            conflits = sum(1 for v in graphe[noeud_courant]
                        if v in solution and solution[v] == couleur)
            eta = 1.0 / (1 + conflits) if conflits > 0 else 2.0

            # Somme des phéromones
            tau = sum(pheromones[noeud_courant][v]
                   for v in graphe[noeud_courant] if v in solution)

            probabilites.append((tau ** alpha) * (eta ** beta))

        # Choix de la couleur
        total = sum(probabilites)
        if total == 0:
            probas_norm = [1.0/len(couleurs)] * len(couleurs)
        else:
            probas_norm = [p/total for p in probabilites]

        couleur_choisie = random.choices(couleurs, weights=probas_norm)[0]
        solution[noeud_courant] = couleur_choisie

    return solution, max_couleurs

def evaluer_solution(solution, graphe):
    """Évalue une solution (conflits et nombre de couleurs)"""
    conflits = 0
    for u in graphe:
        for v in graphe[u]:
            if solution[u] == solution[v]:
                conflits += 1
    return conflits // 2, max(solution.values()) + 1

def recherche_locale(solution, graphe, max_iter=100):
    """Améliore une solution par recherche locale"""
    solution_courante = solution.copy()
    conflits, nb_couleurs = evaluer_solution(solution_courante, graphe)

    for _ in range(max_iter):
        noeuds_conflits = [
            u for u in graphe
            if any(solution_courante[u] == solution_courante[v] for v in graphe[u])
        ]
        if not noeuds_conflits:
            break

        u = random.choice(noeuds_conflits)
        couleur_actuelle = solution_courante[u]

        # Essayer de changer la couleur
        couleurs_interdites = {solution_courante[v] for v in graphe[u]}
        couleurs_possibles = [c for c in range(nb_couleurs)
                            if c not in couleurs_interdites]

        if couleurs_possibles:
            nouvelle_couleur = random.choice(couleurs_possibles)
            solution_courante[u] = nouvelle_couleur
            nouveaux_conflits, nouvelles_couleurs = evaluer_solution(solution_courante, graphe)

            if nouveaux_conflits < conflits or (
                nouveaux_conflits == conflits and nouvelles_couleurs <= nb_couleurs):
                conflits, nb_couleurs = nouveaux_conflits, nouvelles_couleurs
            else:
                solution_courante[u] = couleur_actuelle

    return solution_courante

def mettre_a_jour_pheromones(pheromones, solutions, graphe, rho):
    """Met à jour les traces de phéromones"""
    # Évaporation
    for u in pheromones:
        for v in pheromones[u]:
            pheromones[u][v] *= (1 - rho)

    # Dépôt par les meilleures solutions
    for sol in solutions:
        conflits, nb_couleurs = evaluer_solution(sol['solution'], graphe)
        qualite = 2.0/nb_couleurs if conflits == 0 else 1.0/(10*conflits*nb_couleurs)

        for u in graphe:
            for v in graphe[u]:
                if sol['solution'][u] != sol['solution'][v]:
                    pheromones[u][v] += qualite

    return pheromones

def aco_coloration(graphe, nb_fourmis=20, nb_iterations=50, alpha=1.0, beta=2.0, rho=0.1):
    """Algorithme principal ACO pour la coloration de graphes"""
    # Chronométrage pour chaque étape
    temps_total_debut = time.time()
    temps_construction = 0
    temps_locale = 0
    temps_evaluation = 0
    temps_maj_pheromones = 0

    pheromones = initialiser_pheromones(graphe)
    meilleure_solution = None
    meilleurs_conflits = float('inf')
    meilleures_couleurs = float('inf')
    historique = []

    for iteration in range(nb_iterations):
        solutions_fourmis = []

        for id_fourmi in range(nb_fourmis):
            # Construction de solution
            t_construction_debut = time.time()
            solution, nb_couleurs = construire_solution(graphe, pheromones, alpha, beta)
            temps_construction += time.time() - t_construction_debut

            # Recherche locale (30% de chance)
            t_locale_debut = time.time()
            if random.random() < 0.3:
                solution = recherche_locale(solution, graphe)
            temps_locale += time.time() - t_locale_debut

            # Évaluation
            t_evaluation_debut = time.time()
            conflits, nb_couleurs = evaluer_solution(solution, graphe)
            temps_evaluation += time.time() - t_evaluation_debut

            # Stockage
            solutions_fourmis.append({
                'id': id_fourmi,
                'solution': solution,
                'conflits': conflits,
                'couleurs': nb_couleurs
            })

            # Mise à jour meilleure solution globale
            if conflits < meilleurs_conflits or (
                conflits == meilleurs_conflits and nb_couleurs < meilleures_couleurs):
                meilleurs_conflits = conflits
                meilleures_couleurs = nb_couleurs
                meilleure_solution = solution.copy()

        # Sélection des meilleures solutions (élitisme)
        solutions_triees = sorted(solutions_fourmis, key=lambda x: (x['conflits'], x['couleurs']))
        elites = solutions_triees[:max(1, int(0.3 * nb_fourmis))]

        # Mise à jour des phéromones
        t_maj_pheromones_debut = time.time()
        pheromones = mettre_a_jour_pheromones(pheromones, elites, graphe, rho)
        temps_maj_pheromones += time.time() - t_maj_pheromones_debut

        # Affichage des résultats
        print(f"Itération {iteration + 1}:")
        print(f"Meilleure solution: {meilleurs_conflits} conflits, {meilleures_couleurs} couleurs")
        for sol in solutions_triees[:3]:  # Affiche les 3 meilleures fourmis
            print(f"Fourmi {sol['id']}: {sol['conflits']} conflits, {sol['couleurs']} couleurs")

        # Historique pour analyse
        historique.append({
            'iteration': iteration,
            'meilleur_conflits': meilleurs_conflits,
            'meilleures_couleurs': meilleures_couleurs,
            'solutions': solutions_fourmis
        })

        # Critère d'arrêt
        if meilleurs_conflits == 0:
            print("Solution optimale trouvée!")
            break

    # Calcul du temps total
    temps_total = time.time() - temps_total_debut

    # Affichage des temps d'exécution
    print("\n=== TEMPS D'EXÉCUTION ===")
    print(f"Temps total: {temps_total:.4f} secondes")
    print(f"Temps de construction des solutions: {temps_construction:.4f} secondes ({(temps_construction/temps_total)*100:.1f}%)")
    print(f"Temps de recherche locale: {temps_locale:.4f} secondes ({(temps_locale/temps_total)*100:.1f}%)")
    print(f"Temps d'évaluation: {temps_evaluation:.4f} secondes ({(temps_evaluation/temps_total)*100:.1f}%)")
    print(f"Temps de mise à jour des phéromones: {temps_maj_pheromones:.4f} secondes ({(temps_maj_pheromones/temps_total)*100:.1f}%)")
    print(f"Temps autres opérations: {temps_total - (temps_construction + temps_locale + temps_evaluation + temps_maj_pheromones):.4f} secondes ({(temps_total - (temps_construction + temps_locale + temps_evaluation + temps_maj_pheromones))/temps_total*100:.1f}%)")

    return meilleure_solution, meilleurs_conflits, meilleures_couleurs, historique, {
        'temps_total': temps_total,
        'temps_construction': temps_construction,
        'temps_locale': temps_locale,
        'temps_evaluation': temps_evaluation,
        'temps_maj_pheromones': temps_maj_pheromones
    }

# Exemple d'utilisation
if __name__ == "__main__":
    # Mesure du temps de lecture du graphe
    debut_lecture = time.time()
    # Lire le graphe
    graphe = lire_graphe("/content/r250.5.col.txt")  # Remplacez par votre fichier
    temps_lecture = time.time() - debut_lecture
    print(f"Temps de lecture du graphe: {temps_lecture:.4f} secondes")

    # Informations sur le graphe
    nb_sommets = len(graphe)
    nb_aretes = sum(len(voisins) for voisins in graphe.values()) // 2
    print(f"Taille du graphe: {nb_sommets} sommets, {nb_aretes} arêtes")

    # Exécuter ACO
    print("Démarrage de l'algorithme ACO...")
    solution, conflits, couleurs, historique, temps = aco_coloration(graphe)

    # Afficher les résultats finaux
    print("\n=== RÉSULTATS FINAUX ===")
    print(f"Nombre de conflits: {conflits}")
    print(f"Nombre de couleurs utilisées: {couleurs}")

    # Afficher un échantillon des solutions
    print("\n=== SOLUTIONS DES FOURMIS (échantillon) ===")
    if len(historique) > 1:  # S'il y a plusieurs itérations
        iterations_a_afficher = [0, len(historique)//2, -1]
    else:  # Si une seule itération
        iterations_a_afficher = [0]

    for it in iterations_a_afficher:
        data = historique[it]
        print(f"\nItération {data['iteration'] + 1}:")
        # Afficher des fourmis différentes pour variété
        for sol in data['solutions'][:min(4, len(data['solutions']))]:
            print(f"Fourmi {sol['id']}: {sol['conflits']} conflits, {sol['couleurs']} couleurs")

Temps de lecture du graphe: 0.0205 secondes
Taille du graphe: 250 sommets, 14849 arêtes
Démarrage de l'algorithme ACO...
Itération 1:
Meilleure solution: 0 conflits, 67 couleurs
Fourmi 1: 0 conflits, 67 couleurs
Fourmi 2: 0 conflits, 67 couleurs
Fourmi 3: 0 conflits, 67 couleurs
Solution optimale trouvée!

=== TEMPS D'EXÉCUTION ===
Temps total: 5.0699 secondes
Temps de construction des solutions: 4.9623 secondes (97.9%)
Temps de recherche locale: 0.0176 secondes (0.3%)
Temps d'évaluation: 0.0399 secondes (0.8%)
Temps de mise à jour des phéromones: 0.0467 secondes (0.9%)
Temps autres opérations: 0.0035 secondes (0.1%)

=== RÉSULTATS FINAUX ===
Nombre de conflits: 0
Nombre de couleurs utilisées: 67

=== SOLUTIONS DES FOURMIS (échantillon) ===

Itération 1:
Fourmi 0: 0 conflits, 68 couleurs
Fourmi 1: 0 conflits, 67 couleurs
Fourmi 2: 0 conflits, 67 couleurs
Fourmi 3: 0 conflits, 67 couleurs


#2. COLONIES DE FOURMIS (Algorithme 2)

In [ ]:
import random
import numpy as np
import networkx as nx
from typing import List, Tuple, Set, Dict

In [ ]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time

class AntColonySystemGraphColoring:
    def __init__(self, graph, num_ants=10, alpha=1.0, beta=2.0, rho=0.1, q0=0.9, initial_pheromone=0.1, max_iterations=100):
        """
        Initialize the Ant Colony System for Graph Coloring.

        Parameters:
        - graph: NetworkX graph object
        - num_ants: Number of ants in the colony
        - alpha: Importance of pheromone trails
        - beta: Importance of heuristic information
        - rho: Pheromone evaporation rate
        - q0: Parameter for exploration vs exploitation trade-off (0 <= q0 <= 1)
        - initial_pheromone: Initial pheromone level τ₀
        - max_iterations: Maximum number of iterations
        """
        self.graph = graph
        self.nodes = list(graph.nodes())
        self.num_nodes = len(self.nodes)
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q0 = q0  # Exploration/exploitation parameter
        self.initial_pheromone = initial_pheromone  # τ₀
        self.max_iterations = max_iterations

        # Use a dictionary for edge-based pheromone storage
        self.pheromone = {}
        # Initialize pheromones for node ordering preference
        for u in self.nodes:
            for v in self.nodes:
                if u != v:
                    self.pheromone[(u, v)] = self.initial_pheromone  # Pheromone for selecting v after u

        # Store node indices for faster lookups
        self.node_to_index = {node: idx for idx, node in enumerate(self.nodes)}

        # Initialize node degrees for heuristic information
        self.node_degrees = {node: graph.degree(node) for node in self.nodes}

        # Create extended candidate lists for each node (neighbors and second-order neighbors)
        self.candidate_lists = {node: self.build_extended_candidate_list(node) for node in self.nodes}

        # Best solution found so far
        self.best_coloring = None
        self.best_ordering = None
        self.best_num_colors = float('inf')
        self.global_best_coloring = None
        self.global_best_ordering = None
        self.global_best_num_colors = float('inf')

    def build_extended_candidate_list(self, node):
        """
        Build an extended candidate list including neighbors and second-order neighbors.

        Parameters:
        - node: The node to build a candidate list for

        Returns:
        - List of candidate nodes (neighbors and second-order neighbors)
        """
        candidates = set(self.graph.neighbors(node))  # First-order neighbors

        # Add second-order neighbors
        for neighbor in self.graph.neighbors(node):
            for second_neighbor in self.graph.neighbors(neighbor):
                if second_neighbor != node and second_neighbor not in candidates:
                    candidates.add(second_neighbor)

        return list(candidates)

    def get_pheromone(self, node1, node2):
        """
        Get pheromone value between two nodes.

        Parameters:
        - node1: First node
        - node2: Second node

        Returns:
        - Pheromone value
        """
        return self.pheromone.get((node1, node2), self.initial_pheromone)

    def update_pheromone(self, node1, node2, value):
        """
        Update pheromone value between two nodes.

        Parameters:
        - node1: First node
        - node2: Second node
        - value: New pheromone value
        """
        self.pheromone[(node1, node2)] = value

    def get_valid_colors(self, node, colored_nodes, coloring):
        """
        Return valid colors for a node based on its neighbors' colors.

        Parameters:
        - node: Node to be colored
        - colored_nodes: Set of nodes already colored
        - coloring: Dictionary mapping nodes to colors

        Returns:
        - The smallest valid color for this node
        """
        neighbor_colors = set()
        for neighbor in self.graph.neighbors(node):
            if neighbor in colored_nodes:
                neighbor_colors.add(coloring[neighbor])

        # Find the smallest valid color
        color = 0
        while color in neighbor_colors:
            color += 1

        return color

    def color_graph(self, ordering):
        """
        Color the graph using a given node ordering and greedy approach.

        Parameters:
        - ordering: List of nodes in the order to color them

        Returns:
        - coloring: Dictionary mapping nodes to colors
        - num_colors: Number of colors used
        """
        coloring = {}
        colored_nodes = set()

        for node in ordering:
            color = self.get_valid_colors(node, colored_nodes, coloring)
            coloring[node] = color
            colored_nodes.add(node)

        num_colors = max(coloring.values()) + 1 if coloring else 0
        return coloring, num_colors

    def calculate_saturation_degree(self, node, colored_nodes, coloring):
        """
        Calculate saturation degree (number of differently colored neighbors).

        Parameters:
        - node: Node to calculate saturation degree for
        - colored_nodes: Set of nodes already colored
        - coloring: Dictionary mapping nodes to colors

        Returns:
        - Saturation degree
        """
        colors_used = set()
        for neighbor in self.graph.neighbors(node):
            if neighbor in colored_nodes:
                colors_used.add(coloring[neighbor])
        return len(colors_used)

    def calculate_node_weights(self, current_node, uncolored_nodes, colored_nodes, coloring):
        """
        Calculate weights for choosing the next node to color.

        Parameters:
        - current_node: Current node
        - uncolored_nodes: List of uncolored nodes
        - colored_nodes: Set of nodes already colored
        - coloring: Dictionary mapping nodes to colors

        Returns:
        - Dictionary of weights for nodes in the candidate list
        """
        weights = {}

        # First check candidate list (neighbors and second-order neighbors)
        candidates = [n for n in self.candidate_lists[current_node] if n in uncolored_nodes]

        # If no candidates in the list, use all uncolored nodes
        if not candidates:
            candidates = uncolored_nodes

        for candidate in candidates:
            # Calculate saturation degree (dynamic heuristic)
            sat_degree = self.calculate_saturation_degree(candidate, colored_nodes, coloring)

            # Higher saturation = more constrained = higher priority
            heuristic_value = 1.0 + sat_degree  # Add 1 to avoid division by zero

            # Add degree information to break ties
            heuristic_value *= (1 + 0.1 * self.node_degrees[candidate])

            # Get pheromone value for ordering
            pheromone_value = self.get_pheromone(current_node, candidate)

            weights[candidate] = (pheromone_value ** self.alpha) * (heuristic_value ** self.beta)

        return weights

    def select_next_node(self, current_node, uncolored_nodes, colored_nodes, coloring):
        """
        Select next node using ACS transition rule.

        Parameters:
        - current_node: Current node
        - uncolored_nodes: List of uncolored nodes
        - colored_nodes: Set of nodes already colored
        - coloring: Dictionary mapping nodes to colors

        Returns:
        - Selected node
        """
        if not uncolored_nodes:
            return None

        # Calculate weights
        weights = self.calculate_node_weights(current_node, uncolored_nodes, colored_nodes, coloring)

        # Generate random value for exploration/exploitation decision
        q = random.random()

        if q <= self.q0:  # Exploitation (choose best)
            if weights:
                return max(weights.items(), key=lambda x: x[1])[0]
            else:
                return random.choice(uncolored_nodes)
        else:  # Exploration (probabilistic choice)
            total_weight = sum(weights.values())

            if total_weight == 0:
                return random.choice(uncolored_nodes)

            # Calculate probabilities
            probabilities = [weights[node] / total_weight for node in weights.keys()]

            # Select node based on probabilities
            selected_node = random.choices(list(weights.keys()), weights=probabilities, k=1)[0]
            return selected_node

    def local_pheromone_update(self, node1, node2):
        """
        Update pheromone trail locally when an ant moves from node1 to node2.

        Parameters:
        - node1: First node
        - node2: Second node
        """
        old_value = self.get_pheromone(node1, node2)
        new_value = (1 - self.rho) * old_value + self.rho * self.initial_pheromone
        self.update_pheromone(node1, node2, new_value)

    def global_pheromone_update(self, best_coloring, best_ordering, best_num_colors):
        """
        Update pheromone trails globally based on the best solution in current iteration.

        Parameters:
        - best_coloring: Best coloring found in this iteration
        - best_ordering: Order of nodes in the best solution
        - best_num_colors: Number of colors used in best solution
        """
        # Calculate quality of solution (inverse of number of colors)
        quality = 1.0 / best_num_colors

        # Reward the ordering that led to good coloring
        for i in range(len(best_ordering) - 1):
            node1 = best_ordering[i]
            node2 = best_ordering[i + 1]

            old_value = self.get_pheromone(node1, node2)
            new_value = (1 - self.rho) * old_value + self.rho * quality
            self.update_pheromone(node1, node2, new_value)

        # Also update for same-color nodes that are not adjacent
        # This rewards the formation of good independent sets (color classes)
        for i, node1 in enumerate(self.nodes):
            for j, node2 in enumerate(self.nodes):
                if i < j and best_coloring.get(node1) == best_coloring.get(node2):
                    # If nodes have same color but are not adjacent
                    if not self.graph.has_edge(node1, node2):
                        # Check if they are within 2 hops (have common neighbors)
                        common_neighbors = set(self.graph.neighbors(node1)) & set(self.graph.neighbors(node2))
                        if common_neighbors:
                            old_value = self.get_pheromone(node1, node2)
                            new_value = (1 - self.rho) * old_value + self.rho * quality * 0.5  # Lower reward
                            self.update_pheromone(node1, node2, new_value)

    def construct_solution_for_ant(self, ant_id):
        """
        Construct a graph coloring solution for an ant using ACS rules.

        Parameters:
        - ant_id: Identifier for the ant

        Returns:
        - coloring: Dictionary mapping nodes to colors
        - num_colors: Number of colors used
        - node_ordering: Order in which nodes were colored
        """
        # Choose a starting node (biased toward high-degree nodes)
        probabilities = [self.node_degrees[node] for node in self.nodes]
        total = sum(probabilities)
        if total > 0:
            probabilities = [p / total for p in probabilities]
            current_node = random.choices(self.nodes, weights=probabilities, k=1)[0]
        else:
            current_node = random.choice(self.nodes)

        # Start with all nodes uncolored
        uncolored_nodes = list(self.nodes)
        uncolored_nodes.remove(current_node)
        colored_nodes = {current_node}

        # Initialize partial coloring
        partial_coloring = {current_node: 0}

        node_ordering = [current_node]

        # Select nodes one by one based on ACS transition rule
        while uncolored_nodes:
            next_node = self.select_next_node(current_node, uncolored_nodes, colored_nodes, partial_coloring)

            if next_node:
                # Apply local pheromone update
                self.local_pheromone_update(current_node, next_node)

                # Update partial coloring
                color = self.get_valid_colors(next_node, colored_nodes, partial_coloring)
                partial_coloring[next_node] = color

                # Update state
                node_ordering.append(next_node)
                uncolored_nodes.remove(next_node)
                colored_nodes.add(next_node)
                current_node = next_node

        # Color the graph using the generated ordering
        coloring, num_colors = self.color_graph(node_ordering)

        return coloring, num_colors, node_ordering

    def run(self, verbose=True):
        """
        Run the ACS algorithm to find a good graph coloring.

        Parameters:
        - verbose: Whether to print progress information

        Returns:
        - global_best_coloring: Dictionary mapping nodes to colors
        - global_best_num_colors: Number of colors used
        """
        start_time = time.time()

        for iteration in range(self.max_iterations):
            if verbose and (iteration % 5 == 0 or iteration == self.max_iterations - 1):
                print(f"Iteration {iteration + 1}/{self.max_iterations}")

            # Reset iteration best
            self.best_coloring = None
            self.best_ordering = None
            self.best_num_colors = float('inf')

            # Each ant constructs a solution
            for ant in range(self.num_ants):
                coloring, num_colors, node_ordering = self.construct_solution_for_ant(ant)

                # Update iteration best solution if needed
                if num_colors < self.best_num_colors:
                    self.best_coloring = coloring.copy()
                    self.best_ordering = node_ordering.copy()
                    self.best_num_colors = num_colors

                # Update global best solution if needed
                if num_colors < self.global_best_num_colors:
                    self.global_best_coloring = coloring.copy()
                    self.global_best_ordering = node_ordering.copy()
                    self.global_best_num_colors = num_colors
                    if verbose:
                        print(f"  Found better solution with {self.global_best_num_colors} colors")

            # Apply global pheromone update using the best solution of this iteration
            self.global_pheromone_update(self.best_coloring, self.best_ordering, self.best_num_colors)

            # Optional: report progress every 5 iterations
            if verbose and iteration % 5 == 0:
                elapsed = time.time() - start_time
                print(f"  Time elapsed: {elapsed:.2f}s, Best solution: {self.global_best_num_colors} colors")

        elapsed = time.time() - start_time
        if verbose:
            print(f"\nFinal result: {self.global_best_num_colors} colors in {elapsed:.2f} seconds")

        return self.global_best_coloring, self.global_best_num_colors

    def is_valid_coloring(self):
        """
        Check if the best coloring found is valid.

        Returns:
        - Boolean indicating if coloring is valid
        - List of conflicts (if any)
        """
        if self.global_best_coloring is None:
            return False, ["No solution found yet"]

        conflicts = []
        for u, v in self.graph.edges():
            if self.global_best_coloring[u] == self.global_best_coloring[v]:
                conflicts.append((u, v))

        return len(conflicts) == 0, conflicts

# Helper function to read DIMACS format graph files
def read_dimacs_file(filepath):
    """
    Read a DIMACS format graph file.

    Parameters:
    - filepath: Path to the DIMACS file

    Returns:
    - NetworkX graph object
    """
    G = nx.Graph()
    nodes_added = set()

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith('c'):
                continue

            # Problem line: p FORMAT NODES EDGES
            if line.startswith('p'):
                parts = line.split()
                num_nodes = int(parts[2])
                # Add all nodes to the graph
                for i in range(1, num_nodes + 1):
                    G.add_node(i)
                    nodes_added.add(i)

            # Edge line: e NODE1 NODE2
            elif line.startswith('e'):
                parts = line.split()
                node1 = int(parts[1])
                node2 = int(parts[2])

                # Ensure nodes exist
                if node1 not in nodes_added:
                    G.add_node(node1)
                    nodes_added.add(node1)
                if node2 not in nodes_added:
                    G.add_node(node2)
                    nodes_added.add(node2)

                # Add edge
                G.add_edge(node1, node2)

    return G

In [ ]:
def main():
    # Example usage with a small graph
    # Create a sample graph or read from file
    try:
        dimacs_file = '/content/r250.5.col.txt'
        print(f"Reading graph from {dimacs_file}")
        G = read_dimacs_file(dimacs_file)
    except:
        # Create a sample graph (Petersen graph)
        print("Using sample Petersen graph")
        G = nx.petersen_graph()

    print(f"Graph has {len(G.nodes())} nodes and {len(G.edges())} edges")

    # Set parameters
    num_nodes = len(G.nodes())
    num_ants = min(10, max(2, num_nodes // 5))
    alpha = 2.0      # Importance of pheromone
    beta = 1.0       # Importance of heuristic information
    rho = 0.4        # Pheromone evaporation rate
    q0 = 0.5         # Exploration/exploitation parameter
    initial_pheromone = 0.1
    max_iterations = 50

    print(f"Running ACS with parameters: num_ants={num_ants}, alpha={alpha}, beta={beta}, "
          f"rho={rho}, q0={q0}, tau0={initial_pheromone}, max_iterations={max_iterations}")

    # Create and run Ant Colony System
    acs_algorithm = AntColonySystemGraphColoring(
        G, num_ants, alpha, beta, rho, q0, initial_pheromone, max_iterations
    )
    best_coloring, best_num_colors = acs_algorithm.run()

    # Check if coloring is valid
    valid, conflicts = acs_algorithm.is_valid_coloring()
    if valid:
        print("\nThe coloring is valid!")
    else:
        print("\nWARNING: The coloring is NOT valid!")
        print(f"Found {len(conflicts)} conflicts:")
        for u, v in conflicts[:5]:  # Show only first 5 conflicts
            print(f"Adjacent nodes {u} and {v} both have color {best_coloring[u]}")

    # Calculate statistics
    color_counts = {}
    for color in best_coloring.values():
        if color not in color_counts:
            color_counts[color] = 0
        color_counts[color] += 1

    print("\nColor distribution:")
    for color, count in sorted(color_counts.items()):
        print(f"Color {color}: {count} nodes ({count/len(G.nodes())*100:.1f}%)")

if __name__ == "__main__":
    main()

Reading graph from /content/r250.5.col.txt
Graph has 250 nodes and 14849 edges
Running ACS with parameters: num_ants=10, alpha=2.0, beta=1.0, rho=0.4, q0=0.5, tau0=0.1, max_iterations=50
Iteration 1/50
  Found better solution with 76 colors
  Found better solution with 74 colors
  Found better solution with 72 colors
  Found better solution with 71 colors
  Time elapsed: 3.07s, Best solution: 71 colors
  Found better solution with 70 colors
Iteration 6/50
  Time elapsed: 19.46s, Best solution: 70 colors
Iteration 11/50
  Time elapsed: 37.02s, Best solution: 70 colors
Iteration 16/50
  Time elapsed: 54.06s, Best solution: 70 colors
Iteration 21/50
  Time elapsed: 72.37s, Best solution: 70 colors
Iteration 26/50
  Time elapsed: 89.80s, Best solution: 70 colors
Iteration 31/50
  Time elapsed: 106.31s, Best solution: 70 colors
  Found better solution with 69 colors
Iteration 36/50
  Time elapsed: 123.11s, Best solution: 69 colors
Iteration 41/50
  Time elapsed: 140.38s, Best solution: 69 c